In [2]:
#imports
import os
import logging
import requests
import pandas as pd
from bs4 import BeautifulSoup
from entsoe import EntsoePandasClient
from datetime import datetime
import configparser
from concurrent.futures import ThreadPoolExecutor
import numpy as np

In [3]:
# Start- und Endjahr für die Datenabfrage (inklusive Startjahr, exklusive Endjahr)
# Daten sind nur von 2019 bis 2025 verfügbar
startyear = 2019
endyear = 2025

# Logging konfigurieren
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

def load_config():
    config = configparser.ConfigParser()
    config.read('config.ini')
    config = {
        "api_key": config["API"]["api_key"],
    }
    return config

# API-Key aus der Konfigurationsdatei laden
api_key = load_config()["api_key"]
if not api_key:
    logger.error("API-Schlüssel für ENTSO-E nicht gefunden.")
    exit(1)

client = EntsoePandasClient(api_key=api_key)

# Funktion zum Abrufen der Wetterdaten
def gettable(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'class': 'uk-table uk-table-condensed table_extremwerte'})
        if not table:
            raise ValueError("Tabelle nicht gefunden.")

        rows = table.find_all('tr')
        data = []
        for row in rows[1:]:
            cols = row.find_all('td')
            data.append([ele.text.strip() for ele in cols])

        columns = ['Datum', 'MinTemp', 'MaxTemp', 'Durchschnitt', 'Niederschlag', 'Wind', 'Sonnenstunden', 'NachtTemp']
        return pd.DataFrame(data, columns=columns)
    except Exception as e:
        logger.error(f"Fehler beim Abrufen der Wetterdaten von {url}: {e}")
        return None

# Funktion, um DataFrame auf Stundenbasis zu erweitern
def expand_to_hours(df):
    expanded_data = []
    for _, row in df.iterrows():
        datum = pd.to_datetime(row['Datum'], format='%d.%m.%Y')
        min_temp = float(row['MinTemp'])
        max_temp = float(row['MaxTemp'])

        for hour in range(24):
            timestamp = datum + pd.Timedelta(hours=hour)
            estimated_temp = min_temp + (max_temp - min_temp) * np.sin(np.pi * (hour - 5) / 24)**2
            expanded_data.append({
                'Zeitstempel': timestamp,
                'Geschätzte Temperatur': estimated_temp,
                'Datum': row['Datum'],
                'MinTemp': row['MinTemp'],
                'MaxTemp': row['MaxTemp'],
                'Durchschnitt': row['Durchschnitt']
            })
    return pd.DataFrame(expanded_data)

# Funktion zum Abrufen der Strompreisdaten
def fetch_electricity_prices(year):
    startdate = f'{year}-01-01'
    enddate = f'{year+1}-01-01'

    start = pd.Timestamp(startdate, tz='Europe/Brussels')
    end = pd.Timestamp(enddate, tz='Europe/Brussels')

    try:
        day_ahead_prices = client.query_day_ahead_prices('DE_LU', start=start, end=end)
        day_ahead_prices_df = day_ahead_prices.reset_index()
        day_ahead_prices_df.columns = ['Datum_Uhrzeit', 'Day_ahead_prices']
        day_ahead_prices_df['Strompreis'] = (day_ahead_prices_df['Day_ahead_prices'] / 1000) / 0.3
        day_ahead_prices_df.drop(columns=['Day_ahead_prices'], axis=1, inplace=True)
        return day_ahead_prices_df
    except Exception as e:
        logger.error(f"Fehler beim Abrufen der Strompreise für {year}: {e}")
        return pd.DataFrame()

# Abruf der Wetterdaten
def fetch_weather_data(year):
    all_years_data = []
    for month in range(1, 13):
        start_date = f"{year}-{month:02d}-01"
        end_date = pd.Timestamp(f"{year}-{month+1:02d}-01") - pd.Timedelta(days=1) if month < 12 else pd.Timestamp(f"{year+1}-01-01") - pd.Timedelta(days=1)

        url = f"https://www.wetterkontor.de/de/wetter/deutschland/rueckblick.asp?id=55&datum0={start_date}&datum1={end_date}&jr={year}&mo={month}&datum=08.01.2025&t=4&part=1"

        df_month = gettable(url)
        if df_month is not None:
            df_month = df_month.drop(columns=['Niederschlag', 'Wind', 'Sonnenstunden', 'NachtTemp'])
            df_month.sort_values(by='Datum', inplace=True)
            all_years_data.append(df_month)

    return pd.concat(all_years_data, ignore_index=True) if all_years_data else pd.DataFrame()

# Funktion zum parallelen Abrufen von Wetter- und Strompreisdaten
def fetch_all_data():
    all_weather_data = []
    all_electricity_data = []

    with ThreadPoolExecutor() as executor:
        weather_future = {executor.submit(fetch_weather_data, year): year for year in range(startyear, endyear)}
        electricity_future = {executor.submit(fetch_electricity_prices, year): year for year in range(startyear, endyear)}

        # Ergebnisse abwarten
        for future in weather_future:
            try:
                result = future.result()  # Call result on the future itself
                if not result.empty:
                    all_weather_data.append(result)
            except Exception as e:
                logger.error(f"Fehler beim Abrufen der Wetterdaten für Jahr {weather_future[future]}: {e}")

        for future in electricity_future:
            try:
                result = future.result()  # Call result on the future itself
                if not result.empty:
                    all_electricity_data.append(result)
            except Exception as e:
                logger.error(f"Fehler beim Abrufen der Strompreisdaten für Jahr {electricity_future[future]}: {e}")

    return pd.concat(all_weather_data, ignore_index=True) if all_weather_data else pd.DataFrame(), pd.concat(all_electricity_data, ignore_index=True) if all_electricity_data else pd.DataFrame()

# Hauptlogik
final_weather_data, final_electricity_data = fetch_all_data()

if not final_weather_data.empty and not final_electricity_data.empty:
    # Entfernen von Zeilen ohne gültiges Datum
    final_weather_data = final_weather_data.dropna(subset=['Datum'])
    final_weather_data = final_weather_data[final_weather_data['Datum'].notna() & (final_weather_data['Datum'] != "")]

    # Entfernen von Einheiten und Umwandeln der Temperaturwerte
    final_weather_data['MinTemp'] = final_weather_data['MinTemp'].str.replace(',', '.').astype(float)
    final_weather_data['MaxTemp'] = final_weather_data['MaxTemp'].str.replace(',', '.').astype(float)

    expanded_weather_df = expand_to_hours(final_weather_data)
    expanded_weather_df.drop(columns=['Datum', 'MinTemp', 'MaxTemp', 'Durchschnitt'], inplace=True)

    expanded_weather_df['Zeitstempel'] = pd.to_datetime(expanded_weather_df['Zeitstempel'], errors='coerce', utc=True).dt.tz_localize(None)
    final_electricity_data['Datum_Uhrzeit'] = pd.to_datetime(final_electricity_data['Datum_Uhrzeit'], errors='coerce', utc=True).dt.tz_localize(None)

    # Merge und Füllen der fehlenden Strompreise
    merged_df = pd.merge(expanded_weather_df, final_electricity_data, left_on='Zeitstempel', right_on='Datum_Uhrzeit', how='left')
    merged_df.drop(columns=['Datum_Uhrzeit'], inplace=True)
    merged_df['Strompreis'] = merged_df['Strompreis'].fillna(method='ffill').fillna(method='bfill')

    # Entfernen von Duplikaten und umbennenen des dataframes
    Wetter_und_Strompreis_df = merged_df.drop_duplicates(keep='first')
else:
    logger.error("Fehler: Wetter- oder Strompreisdaten konnten nicht abgerufen werden.")

# Daten aus CSV-Datei lesen
file_path = 'Energiesystem.csv'  # Pfad zur CSV-Datei
df_basisModel = pd.read_csv(file_path)
# Spalte unamed in Zeitstempel umbenennen
df_basisModel.rename(columns={'Unnamed: 0': 'Zeitstempel'}, inplace=True)
# Spalte Zeitstempel in Datetime umwandeln
df_basisModel['Zeitstempel'] = pd.to_datetime(df_basisModel['Zeitstempel'], errors='coerce', utc=True).dt.tz_localize(None)

# Daten aus Basis-Modell Wetter und Strom  joinen,
df_2019 = pd.merge(df_basisModel, merged_df, on='Zeitstempel', how='left')
# Duplikate entfernen
df_2019 = df_2019.drop_duplicates(keep='first')
# DataFrame anzeigenprint(df_2019)

2025-01-12 20:03:22,843 - WARNING - Connection Error, retrying in 0 seconds
2025-01-12 20:03:22,845 - WARNING - Connection Error, retrying in 0 seconds
2025-01-12 20:03:22,846 - ERROR - Fehler beim Abrufen der Strompreise für 2023: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
2025-01-12 20:03:22,847 - ERROR - Fehler beim Abrufen der Strompreise für 2022: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
C:\Users\Julia\AppData\Local\Temp\ipykernel_20956\3190073618.py:153: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df['Strompreis'] = merged_df['Strompreis'].fillna(method='ffill').fillna(method='bfill')


In [4]:
# DataFrame als Pickle-Datei speichern
df_2019.to_pickle('daten.pkl')
